In [12]:
from IPython.display import Markdown  as md
from  matplotlib import pyplot as plt
import sqlite3  
import pandas as pd
import numpy as np
import json
from copy import deepcopy
from datetime import datetime

In [13]:
df=pd.read_csv('tmall_order_report.csv')
df=df.where(pd.notnull(df),None)
df['订单创建日期']=df['订单创建时间'].apply(lambda x:pd.Timestamp(x).date()).astype(str)


# 将数据写入example_table 中
with sqlite3.connect('example.db') as conn:
    df.to_sql('example_table',con=conn,if_exists='replace',index=False)
    
connection=sqlite3.connect('example.db')

In [14]:
df.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额',
       '订单创建日期'],
      dtype='object')

In [15]:
df.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额,订单创建日期
0,1,178.8,0.0,上海,2020-02-21 00:00:00,None,0.0,2020-02-21
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0,2020-02-20
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,None,0.0,2020-02-20
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0,2020-02-20
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8,2020-02-20


In [16]:
query=""" 
create view
if not exists
view_1
as 
select 
    '订单创建时间' as create_time,
    '收货地址' as address
from example_table

"""
result=connection.execute(query).fetchall()
result

[]

In [17]:
# 删除view
query=""" 
drop view
if exists
view_1

"""
result=connection.execute(query).fetchall()
result

[]

In [18]:
query=''' 

select 
    t1.order_date,
    sum(orders_num)
    over(
        order by order_date
        rows 
        between 
        unbounded preceding
        and 
        current row
    )as running_total
from 
(
    select 
    "订单创建日期" as order_date,
    count("订单编号") as order_num
    from example_table
    group by "订单创建日期"
)as t1
        
'''

# 加入 VIEW
query_view = '''
CREATE VIEW IF NOT EXISTS orders_by_date AS
SELECT 
    "订单创建日期" AS order_date
    ,COUNT("订单编号") AS orders_num
FROM example_table
GROUP BY "订单创建日期";

SELECT
      order_date,
      SUM(orders_num)                  -- 聚合函数 SUM()
      OVER(                            -- OVER 关键字，（）内定 frame
             ORDER BY order_date        -- 按照 order_date 排序 
             ROWS                       -- 按照表当中的相对位置
             BETWEEN                    -- 界定上下界
                  UNBOUNDED PRECEDING   -- 上界，不设限制
             AND 
                  CURRENT ROW           -- 下界，当前行
             ) AS running_total

FROM orders_by_date;
'''
### 【！注意】这里将 execute 换为 executescript
### 这样我们可以在一个 query 当中执行多条 SQL 语句
result = connection.executescript(query_view).fetchall() 
result

[]

In [19]:
query=''' 
select order_date,orders_num
from orders_by_date;
'''
result=connection.execute(query).fetchone()
result

('2020-02-01', 176)

In [20]:
# with 函数构建cte
query_cte=""" 
with
orders_by_date_cte
as(
    select
    "订单创建日期" as order_date,
    count("订单编号") as orders_num
    from example_table
    group by order_date
)
select
    order_date,
    orders_num,
    sum(orders_num)
    over(
        order by order_date
        rows 
        between
            unbounded preceding
        and
            current row
    )as running_total
from orders_by_date_cte;
"""
result=connection.execute(query_cte).fetchall()
result

[('2020-02-01', 176, 176),
 ('2020-02-02', 222, 398),
 ('2020-02-03', 267, 665),
 ('2020-02-04', 469, 1134),
 ('2020-02-05', 369, 1503),
 ('2020-02-06', 144, 1647),
 ('2020-02-07', 177, 1824),
 ('2020-02-09', 404, 2228),
 ('2020-02-10', 27, 2255),
 ('2020-02-11', 15, 2270),
 ('2020-02-12', 1, 2271),
 ('2020-02-13', 5, 2276),
 ('2020-02-14', 7, 2283),
 ('2020-02-15', 5, 2288),
 ('2020-02-17', 390, 2678),
 ('2020-02-18', 1015, 3693),
 ('2020-02-19', 1025, 4718),
 ('2020-02-20', 1345, 6063),
 ('2020-02-21', 2068, 8131),
 ('2020-02-22', 2027, 10158),
 ('2020-02-23', 2200, 12358),
 ('2020-02-24', 1998, 14356),
 ('2020-02-25', 3416, 17772),
 ('2020-02-26', 2849, 20621),
 ('2020-02-27', 2586, 23207),
 ('2020-02-28', 2691, 25898),
 ('2020-02-29', 2112, 28010)]

In [21]:
# cte 不会存储
query_cte="""  
select
    order_date,
    sum(orders_num)
    over(
        order by order_date
        rows 
        between 
            unbounded   preceding
        and
            current row
    )as running_total
from orders_by_date_cte;

"""
try:
    result=connection.execute(query).fetchall()
except Exception as e:
    print(e)

In [25]:
query = '''
WITH orders_by_date AS
(
SELECT "订单创建日期" AS order_date
               ,"收货地址 " AS address
               ,COUNT("订单编号") AS orders_num
      FROM example_table
      GROUP BY "订单创建日期","收货地址 "
)
,filter_address AS            -- 新的 CTE
(
SELECT 
    order_date
    ,orders_num 
    ,address
FROM orders_by_date           -- 引用上一个 CTE
WHERE address IN ('北京','上海')
)
SELECT
      t1.order_date
      ,t1.address
      ,SUM(orders_num)                  -- 聚合函数 SUM()
       OVER(                            -- OVER 关键字，（）内定 frame
             PARTITION BY address       -- 每个 frame 内只有 address 相同的数据
             ORDER BY order_date        -- 按照 order_date 排序 
             ROWS                       -- 按照表当中的相对位置
             BETWEEN                    -- 界定上下界
                  UNBOUNDED PRECEDING   -- 上界，不设限制
             AND 
                  CURRENT ROW           -- 下界，当前行
             ) AS running_total

FROM filter_address AS t1
'''
result = connection.execute(query).fetchall()
result[:20]

[('2020-02-01', '上海', 48),
 ('2020-02-02', '上海', 114),
 ('2020-02-03', '上海', 197),
 ('2020-02-04', '上海', 330),
 ('2020-02-05', '上海', 414),
 ('2020-02-06', '上海', 459),
 ('2020-02-07', '上海', 499),
 ('2020-02-09', '上海', 596),
 ('2020-02-10', '上海', 603),
 ('2020-02-11', '上海', 608),
 ('2020-02-13', '上海', 609),
 ('2020-02-14', '上海', 612),
 ('2020-02-17', '上海', 685),
 ('2020-02-18', '上海', 834),
 ('2020-02-19', '上海', 963),
 ('2020-02-20', '上海', 1090),
 ('2020-02-21', '上海', 1291),
 ('2020-02-22', '上海', 1508),
 ('2020-02-23', '上海', 1717),
 ('2020-02-24', '上海', 1957)]